In [2]:
import json

train_json = "/home/t-jiaweiwang/Project/GOT-OCR2.0/datasets/DocLayNet/train_textlines_others_add_text_400_converted_add_in_graphical_others_grouped.json"

train_annotations = json.load(open(train_json))

In [2]:
train_annotations.keys()

dict_keys(['images', 'categories', 'annotations'])

In [3]:
'''
Conversation Format:

{
    "image": "image_path",
    "conversations": [
        {
            "from": "human",
            "value": "<image>\nPOD: "
        },
        {
            "from": "gpt",
            "value": "[Box1] [Logical Role]\n[Box2] [Logical Role]\n..."
        }
    ]
}
'''

conversations = []


'\nConversation Format:\n\n{\n    "image": "image_path",\n    "conversations": [\n        {\n            "from": "human",\n            "value": "<image>\nPOD: "\n        },\n        {\n            "from": "gpt",\n            "value": "[Box1] [Logical Role]\n[Box2] [Logical Role]\n..."\n        }\n    ]\n}\n'

In [5]:
train_annotations['annotations'][1]

{'id': 1,
 'category_id': 4,
 'AxisAlignedBBox': [100.1946343137255,
  117.51495402298849,
  789.325354248366,
  42.370593869731806],
 'bbox': [100.1946343137255,
  117.51495402298849,
  789.325354248366,
  42.370593869731806,
  0],
 'quad_coord': [100.1946343137255,
  117.51495402298849,
  889.5199885620915,
  117.51495402298849,
  889.5199885620915,
  159.8855478927203,
  100.1946343137255,
  159.8855478927203],
 'segmentation': [[100.1946343137255,
   117.51495402298849,
   100.1946343137255,
   159.8855478927203,
   889.5199885620915,
   159.8855478927203,
   889.5199885620915,
   117.51495402298849]],
 'iscrowd': 0,
 'area': 33444.1840159397,
 'tags': ['Text'],
 'tag_curve': 0,
 'tag_notsure': 0,
 'textline_polys': [[100.1946343137255,
   117.51495402298849,
   889.5199885620915,
   117.51495402298849,
   889.5199885620915,
   159.8855478927203,
   100.1946343137255,
   159.8855478927203]],
 'textline_contents': ['Finance receivables that originated outside the U.S. were $52.7 bil

In [5]:
imageid2annotations = {}

for annotation in train_annotations["annotations"]:
    if annotation["image_id"] not in imageid2annotations:
        imageid2annotations[annotation["image_id"]] = []
    imageid2annotations[annotation["image_id"]].append(annotation)

imageid2image = {}
for image in train_annotations["images"]:
    imageid2image[image["id"]] = image

conversations = []
for image_id, annotations in imageid2annotations.items():
    image = imageid2image[image_id]
    image_path = image["file_name"]
    conversation = {
        "image": "train/" + image_path,
        "conversations": []
    }
    conversation["conversations"].append({
        "from": "human",
        "value": "<image>\nPOD: "
    })
    gt_text = ""
    gt_boxes = []
    gt_logical_roles = []
    for annotation in annotations:
        bbox = annotation["quad_coord"][:2] + annotation["quad_coord"][4:6]
        # x = int(x/w * 1000)
        # y = int(y/h * 1000)
        bbox = [int(bbox[0] / image["width"] * 1000), int(bbox[1] / image["height"] * 1000), int(bbox[2] / image["width"] * 1000), int(bbox[3] / image["height"] * 1000)]
        if "others" in annotation["tags"]:
            # exclude others
            continue
        gt_boxes.append(bbox)
        gt_logical_roles.append(annotation["tags"][0])

    # sort the boxes from top to bottom from left to right
    # 将 gt_boxes 和 gt_logical_roles 结合在一起，根据 gt_boxes 进行排序
    sorted_pairs = sorted(zip(gt_boxes, gt_logical_roles), key=lambda x: (x[0][1], x[0][0]))
    if len(sorted_pairs) == 0:
        continue
    # 解压缩，提取排序后的 gt_boxes 和 gt_logical_roles
    gt_boxes, gt_logical_roles = zip(*sorted_pairs)

    # 将它们转换回列表（因为 zip 返回的是元组）
    gt_boxes = list(gt_boxes)
    gt_logical_roles = list(gt_logical_roles)
    
    for bbox, logical_role in zip(gt_boxes, gt_logical_roles):
        gt_text += f"{str(bbox)} {logical_role}\n"
    
    conversation["conversations"].append({
        "from": "gpt",
        "value": gt_text
    })
    conversations.append(conversation)

In [1]:
conversations[0]

NameError: name 'conversations' is not defined

In [7]:
json.dump(conversations, open("/home/t-jiaweiwang/Project/GOT-OCR2.0/datasets/DocLayNet/got_annotations/train_got_conversations_wo_others.json", "w"), indent=4)

In [7]:
# 可视化GT
import json

conversations = json.load(open("/home/t-jiaweiwang/Project/GOT-OCR2.0/datasets/DocLayNet/got_annotations/train_got_conversations_wo_others.json"))
converstation = conversations[0]
image_path = "/home/t-jiaweiwang/Project/GOT-OCR2.0/datasets/DocLayNet/" + converstation["image"]
outputs = converstation["conversations"][1]["value"]
print(outputs)
outputs_list = outputs.split('\n')
# visualize the bboxes and logical role in args.image_file: [117, 88, 328, 100] ['Section-header']
import cv2
image_viz = cv2.imread(image_path)
w, h = image_viz.shape[1], image_viz.shape[0]
for out in outputs_list:
    try:
        bbox, logical_role = out.split('] ')
    except:
        continue
    bbox = bbox[1:].split(', ')
    # bbox = [int(i) for i in bbox]
    bbox = [int(i) / 1000 for i in bbox]
    bbox = [int(bbox[0]*w), int(bbox[1]*h), int(bbox[2]*w), int(bbox[3]*h)]
    if bbox[0] < 0 or bbox[1] < 0 or bbox[2] < 0 or bbox[3] < 0:
        continue
    image_viz = cv2.rectangle(image_viz, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
    image_viz = cv2.putText(image_viz, logical_role, (bbox[0], bbox[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

cv2.imwrite('test_gt.jpg', image_viz)

[70, 54, 433, 74] Page-header
[97, 114, 867, 155] Text
[97, 162, 866, 217] Text
[97, 223, 861, 278] Text
[97, 285, 813, 326] Text
[97, 333, 836, 360] Text
[94, 361, 891, 473] Table
[97, 488, 876, 529] Text
[97, 549, 789, 567] Section-header
[97, 572, 608, 587] Text
[96, 592, 886, 675] Table
[97, 687, 886, 742] Text
[97, 749, 782, 776] Text
[97, 783, 887, 838] Text
[924, 963, 947, 979] Page-footer



True